# Tools and Toolchains

This notebook demonstrates using tools and toolchains when ordering with the orders api. Specifically, this notebook demonstrates the following toolchains:
 - [clip](#clip)
 - [bandmath](#bandmath)
 - [toar](#toar)
 - [composite](#composite)
 - [clip -> bandmath](#clip_bandmath)
 - [toar -> reproject -> tile](#toar_reproject_tile)

For background on ordering and downloading with the orders api, review the REST API + Python training.

Reference information can be found at [Tools & toolchains](https://developers.planet.com/docs/orders/tools-toolchains/).

## Setup

In [ ]:
import json
import os
import pathlib
import time

import numpy as np
import rasterio
from rasterio.plot import show
import requests
from requests.auth import HTTPBasicAuth

In [ ]:
# API Key stored as an env variable
PLANET_API_KEY = os.getenv('PL_API_KEY')
#PLANET_API_KEY = ""
orders_url = 'https://api.planet.com/compute/ops/orders/v2'

# set up requests to work with api
auth = HTTPBasicAuth(PLANET_API_KEY, '')
headers = {'content-type': 'application/json'}

In [ ]:
# define products part of order
single_product = [
    {
      "item_ids": ["20151119_025740_0c74"],
      "item_type": "PSScene",
      "product_bundle": "analytic_udm2"
    }
]

single_product_rgb = [
    {
      "item_ids": ["20151119_025740_0c74"],
      "item_type": "PSScene",
      "product_bundle": "visual"
    }
]

same_src_products = [
    {
      "item_ids": ["20151119_025740_0c74",
                   "20151119_025739_0c74"],
      "item_type": "PSScene",
      "product_bundle": "visual"
    }
]

multi_src_products = [
    {
      "item_ids": ["20151119_025740_0c74"],
      "item_type": "PSScene",
      "product_bundle": "analytic_udm2"
    },
    {
      "item_ids": ["20220628_183020_20_248c"],
      "item_type": "PSScene",
      "product_bundle": "analytic_8b_udm2"
    },
    
]

In [ ]:
# define helpful functions for submitting, polling, and downloading an order
def place_order(request, auth):
    response = requests.post(orders_url, data=json.dumps(request), auth=auth, headers=headers)
    print(response)
    
    if not response.ok:
        raise Exception(response.content)

    order_id = response.json()['id']
    print(order_id)
    order_url = orders_url + '/' + order_id
    return order_url

def poll_for_success(order_url, auth, num_loops=50):
    count = 0
    while(count < num_loops):
        count += 1
        r = requests.get(order_url, auth=auth)
        response = r.json()
        state = response['state']
        print(state)
        success_states = ['success', 'partial']
        if state == 'failed':
            raise Exception(response)
        elif state in success_states:
            break
        
        time.sleep(10)
        
def download_order(order_url, auth, overwrite=False):
    r = requests.get(order_url, auth=auth)
    print(r)

    response = r.json()
    results = response['_links']['results']
    results_urls = [r['location'] for r in results]
    results_names = [r['name'] for r in results]
    results_paths = [pathlib.Path(os.path.join('data', n)) for n in results_names]
    print('{} items to download'.format(len(results_urls)))
    
    for url, name, path in zip(results_urls, results_names, results_paths):
        if overwrite or not path.exists():
            print('downloading {} to {}'.format(name, path))
            r = requests.get(url, allow_redirects=True)
            path.parent.mkdir(parents=True, exist_ok=True)
            open(path, 'wb').write(r.content)
        else:
            print('{} already exists, skipping {}'.format(path, name))
            
    return dict(zip(results_names, results_paths))

## Tool Demos

### No Processing (reference)

We will order and download the unprocessed image for comparison with the output of the toolchains defined below.

In [ ]:
request = {
  "name": "no processing",
  "products": single_product_rgb,
}

In [ ]:
order_url = place_order(request, auth)

### Clip
<a id='clip'></a>

Clipping is likely the most common tool that will be used. It allows us to only download the pixels we are interested in.

In [ ]:
clip_aoi = {
    "type":"Polygon",
    "coordinates":[[[94.81858044862747,15.858073043526062],
                    [94.86242249608041,15.858073043526062],
                    [94.86242249608041,15.894323164978303],
                    [94.81858044862747,15.894323164978303],
                    [94.81858044862747,15.858073043526062]]]
}

In [ ]:
# define the clip tool
clip = {
    "clip": {
        "aoi": clip_aoi
    }
}

In [ ]:
# create an order request with the clipping tool
request_clip = {
  "name": "just clip",
  "products": single_product_rgb,
  "tools": [clip]
}

request_clip

In [ ]:
clip_order_url = place_order(request_clip, auth)

### Band Math
<a id='bandmath'></a>

To demonstrate band math we will order an NDVI image.

In [ ]:
bandmath = {
  "bandmath": {
    "pixel_type": "32R",
    "b1": "(b4 - b3) / (b4 + b3)"
  }
}

In [ ]:
bandmath_request = {
  "name": "band math",
  "products": single_product,
  "tools": [bandmath]
}
bandmath_request

In [ ]:
bandmath_order_url = place_order(bandmath_request, auth)

### TOAR
<a id='toar'></a>

The `toar` tool converts imagery to Top of Atmosphere Reflectance.

In [ ]:
toar = {
      "toar": {
        "scale_factor": 10000
      }
}
toar_request = {
  "name": "toar",
  "products": single_product,
  "tools": [toar]
}
toar_request

In [ ]:
toar_order_url = place_order(toar_request, auth)

### Composite
<a id='composite'></a>

The composite tool combines multiple images into one image, similar to mosaicing. The input images must have the same band configuration, and that band configuration will be propagated to the output image.

In [ ]:
# composite = {  
#    "composite":{ "group_by": "strip_id" 
#    }
# }

composite = {  
   "composite":{ 
   }
}

composite_request = {
  "name": "composite",
  "products": same_src_products,
  "tools": [composite]
}

composite_request

In [ ]:
composite_order_url = place_order(composite_request, auth)

### Combined Tools - Clip and Band Math
<a id='clip_bandmath'></a>

This toolchain demonstrates how we can combine the clipping tool with the NDVI band math tool to only process and download the NDVI values for pixels we are interested in. Combining tools is as simple as combining the tool definitions in a list in the order request.

In [ ]:
clip_bandmath_request = {
  "name": "clip and bandmath",
  "products": single_product,
  "tools": [clip, bandmath]
}
clip_bandmath_request

In [ ]:
clip_bandmath_order_url = place_order(clip_bandmath_request, auth)

## TOAR, Reproject, Tile
<a id='toar_reproject_tile'></a>

For a more complicated example, we will convert the pixels to reflectance, project them to WGS84, and then tile them.

In [ ]:
reproject =     {
    "reproject": {
        "projection": "WGS84",
        "kernel": "cubic"
    }
}

tile = {
  "tile": {
    "tile_size": 1232,
    "origin_x": -180,
    "origin_y": -90,
    "pixel_size": 0.000027056277056,
    "name_template": "C1232_30_30_{tilex:04d}_{tiley:04d}"
  }
}

In [ ]:
trt_request = {
  "name": "toar reproject tile",
  "products": single_product,
  "tools": [toar, reproject, tile]
}
trt_request

In [ ]:
trt_order_url = place_order(trt_request, auth)

## Visualize

In [ ]:
# define helpful functions for visualizing downloaded imagery
def show_analytic(img_file):
    with rasterio.open(img_file) as src:
        b,g,r,n = src.read()

    rgb = np.stack((r,g,b), axis=0)
    show(rgb/rgb.max())
    
def show_rgb(img_file):
    with rasterio.open(img_file) as src:
        show(src)
    
    
def show_gray(img_file):
    with rasterio.open(img_file) as src:
        g = src.read(1)
    show(g/g.max())

#### No processing

In [ ]:
poll_for_success(order_url, auth)
downloaded_files = download_order(order_url, auth)
img_file = next(downloaded_files[d] for d in downloaded_files
                if d.endswith('3B_Visual.tif'))
print(img_file)

In [ ]:
show_rgb(img_file)

#### Clipped

In [ ]:
poll_for_success(clip_order_url, auth)
downloaded_clip_files = download_order(clip_order_url, auth)
clip_img_file = next(downloaded_clip_files[d] for d in downloaded_clip_files
                 if d.endswith('_3B_Visual_clip.tif'))
clip_img_file

In [ ]:
show_rgb(clip_img_file)

#### Bandmath

In [ ]:
poll_for_success(bandmath_order_url, auth)
downloaded_bandmath_files = download_order(bandmath_order_url, auth)
bandmath_img_file = next(downloaded_bandmath_files[d] for d in downloaded_bandmath_files
                         if d.endswith('_bandmath.tif'))
bandmath_img_file

In [ ]:
show_rgb(img_file)
show_gray(bandmath_img_file)

#### TOAR

In [ ]:
poll_for_success(toar_order_url, auth)
downloaded_toar_files = download_order(toar_order_url, auth)
toar_img_file = next(downloaded_toar_files[d] for d in downloaded_toar_files
                 if d.endswith('_toar.tif'))
toar_img_file

In [ ]:
show_rgb(img_file)
show_analytic(toar_img_file)

#### Composite

In [ ]:
poll_for_success(composite_order_url, auth)
downloaded_composite_files = download_order(composite_order_url, auth)
composite_file = next(downloaded_composite_files[d] for d in downloaded_composite_files
                      if d.endswith('composite.tif'))
composite_file

In [ ]:
show_rgb(composite_file)

#### Clip and Bandmath

In [ ]:
poll_for_success(clip_bandmath_order_url, auth)
downloaded_clip_bandmath_files = download_order(clip_bandmath_order_url, auth)
clip_bandmath_file = next(downloaded_clip_bandmath_files[d] for d in downloaded_clip_bandmath_files
                 if d.endswith('_clip_bandmath.tif'))
clip_bandmath_file

In [ ]:
show_gray(clip_bandmath_file)

#### TOAR/Reproject/Tile

In [ ]:
poll_for_success(trt_order_url, auth)
downloaded_trt_files = download_order(trt_order_url, auth)
tile_files = list(d for d in downloaded_trt_files.values()
                 if d.name.startswith('C1232_30_30_'))

In [ ]:
for f in tile_files[:8]:
    show_rgb(f)

In [ ]:
test_file = tile_files[0]

In [ ]:
!gdalinfo $test_file

In [ ]:
!gdalinfo $img_file